In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../data/foodb/Content.csv")

C:\Users\oszud\AppData\Local\Temp\ipykernel_15584\2274376406.py:1: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/foodb/Content.csv")


Original unit of protein mg/100g

In [3]:
data_prot = data[data.orig_source_name == "Protein"]
data_prot.head()

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
399,400,2,Nutrient,4,9148,"Kiwifruit, green, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1140.0,raw,0
400,401,2,Nutrient,4,9445,"Kiwifruit, gold, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1230.0,raw,0
410,411,2,Nutrient,5,2026,"Spices, onion powder",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,10410.0,dried or powder,0
411,412,2,Nutrient,5,11282,"Onions, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1100.0,raw,0
412,413,2,Nutrient,5,11283,"Onions, cooked, boiled, drained, without salt",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1360.0,cooked,0


In [4]:
print("Raw protein info: ", len(data_prot))

Raw protein info:  7336


In [5]:
data_n = data[data.orig_source_name == "Protein, total-N"]
data_n.head()

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
396,397,2,Nutrient,2,674,"Cabbage, savoy, raw",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,300.0,raw,0
398,399,2,Nutrient,4,465,"Kiwi fruit, raw",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,200.0,raw,0
403,404,2,Nutrient,5,148,"Onion, raw",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,300.0,raw,0
405,406,2,Nutrient,5,830,"Onions, chopped and deepfried",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1000.0,cooked,0
407,408,2,Nutrient,5,662,"Onions, chopped, frozen",NaN,NaN,0002,"Protein, total-N",...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,100.0,"raw, frozen",0


As you see - there is also "Protein, total-N" type of records. Those records tell us about nitrogen content. Internet says that sometimes it is used to estimate how much protein is actually in a food. However, because it is not that straightforward - I suggest we do not dig into it right now. If we won't obtain enough info - it might be somehow useful.

Based on 
```
 from ydata_profiling import ProfileReport

 profile = ProfileReport(data_prot, title='Protein FooDB Report')
 profile
```
and
```
 data_prot.COLNAME.unique()
```
we'll drop some columns

In [31]:
data_prot = data[data.orig_source_name == "Protein"]
data_prot = data_prot.drop(columns=['creator_id', 'updater_id', 'created_at', 'updated_at', 
                                    'orig_citation', 'citation', 'citation_type',
                                    'orig_unit', 'source_id', 'orig_source_name', 'orig_source_id',
                                    'orig_food_scientific_name', 'orig_food_part', 'orig_min', 'orig_max',
                                    'orig_method', 'orig_unit_expression', 'source_type'])

In [32]:
data_prot.head()

,id,food_id,orig_food_id,orig_food_common_name,orig_content,standard_content,preparation_type,export
399,400,4,9148,"Kiwifruit, green, raw",1140.0,1140.0,raw,0
400,401,4,9445,"Kiwifruit, gold, raw",1230.0,1230.0,raw,0
410,411,5,2026,"Spices, onion powder",10410.0,10410.0,dried or powder,0
411,412,5,11282,"Onions, raw",1100.0,1100.0,raw,0
412,413,5,11283,"Onions, cooked, boiled, drained, without salt",1360.0,1360.0,cooked,0


In [34]:
data_prot[data_prot['orig_food_common_name'].str.contains('Onions')]

,id,food_id,orig_food_id,orig_food_common_name,orig_content,standard_content,preparation_type,export
411,412,5,11282,"Onions, raw",1100.0,1100.0,raw,0
412,413,5,11283,"Onions, cooked, boiled, drained, without salt",1360.0,1360.0,cooked,0
413,414,5,11284,"Onions, dehydrated flakes",8950.0,8950.0,dried or powder,0
414,415,5,11285,"Onions, canned, solids and liquids",850.0,850.0,NaN,0
415,416,5,11286,"Onions, yellow, sauteed",950.0,950.0,cooked,0
416,417,5,11287,"Onions, frozen, chopped, unprepared",790.0,790.0,"raw, frozen",0
417,418,5,11288,"Onions, frozen, chopped, cooked, boiled, drain...",770.0,770.0,cooked,0
418,419,5,11289,"Onions, frozen, whole, unprepared",890.0,890.0,"raw, frozen",0
419,420,5,11290,"Onions, frozen, whole, cooked, boiled, drained...",710.0,710.0,cooked,0
420,421,5,11291,"Onions, spring or scallions (includes tops and...",1830.0,1830.0,raw,0
